In [2]:
"""
All imports for each of our downstream cells.
"""

# Simulator
from mpm.engine import Engine
from mpm.cube import cube

# Utils
import numpy as np
import os
from typing import Final, List, Tuple, Any

# ML
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras import layers as L
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, History
from tensorflow.keras.losses import Huber
import pickle

# Viz
import matplotlib.pyplot as plt

# Comment this out if printing causes crashes.
np.set_printoptions(suppress=True, threshold=np.inf, linewidth=500, precision=2)


In [ ]:
"""
Some helper constants for the training data
"""

# Number of simulation steps
_TRAIN_DATA_STEPS = 2000

_TRAIN_DATA_GMIN: Final[int] = -300
_TRAIN_DATA_GMAX: Final[int] = -10

# The gravitation increment for the sim
_TRAIN_DATA_GINCR: Final[int] = 10

# Resolutions for the paper - These are doubled due to there being two cubes
_TRAIN_DATA_RESOLUTIONS: Final[List[int]] = [200, 400, 600]

# Coarseness for the paper - The sampling frequency
_TRAIN_DATA_COARSENINGS: Final[List[int]] = [10, 15, 20]

In [ ]:
"""
Some helpter constants for the test data
"""
# Number of simulation steps
_TEST_DATA_STEPS = 3000

# We only run for this gravitation scenario
_TEST_DATA_GMIN: Final[int] = -100
_TEST_DATA_GMAX: Final[int] = -100

# Static coarsening, we use this to not surprise the model _too_ much
_TEST_DATA_COARSENING: Final[int] = 10

In [3]:

"""
Some helpter constants for ML models 
"""

# If you're in a 3D simulation, this value is 4
_CHANNELS: Final[int] = 3

# The shape of the input to the neural network
_INPUT_SHAPE: Final[Tuple[int, int, int]] = (64, 64, _CHANNELS)

# Determined in the paper, your mileage may vary.
_LR: Final[float] = 0.0001

# Vary this value to use a different loss "mae", "mse", "huber" etc.
_LOSS: Final[str] = "huber"

# Exponent value to control the size of the model [Taken directly from Thuerey et. al.]
# 3 - 86k
# 4 - 330k
# 5 - 1.3m
# 6 - 5.4m
# 7 - 21m
_EXPO: Final[int] = 3

# Our dropout perccentage
_DROPOUT: Final[float] = 0.2

# The number of training epochs
_EPOCHS: Final[int] = 200

# Batch size, you may need to change this depending on available gpu memory
_BATCH_SIZE: Final[int] = 25

# Validation Split size
_VALIDATION_SPLIT: Final[float] = 0.1

In [ ]:
"""
Some helper constants for plots
"""

# The y axis labels for the ground truth plots
_Y_AXIS_LABELS_GROUND_TRUTH = ["X-Velocity", "Y-Velocity", "Mask"]

# The output color for the matplotlib plots
_CMAP: Final[str] = "plasm"

In [ ]:
"""
Generate Datasets
"""

# Make the training-set cubes in the simulation
def make_train_sim_object(res: int, material_model: str):
    center = (0.4, 0.6)
    x1 = cube(center=center, res=res)

    center = (0.3, 0.5)
    x2 = cube(center=center, res=res)

    # Move this cube closer to the ground.
    x2[:, 1] -= 0.2

    # Combine the points into one large matrix of points
    cubes = np.concatenate((x1, x2))

    # Set material type
    material = np.array([material_model] * len(cubes))

    return cubes, material


def generate_all_train_datasets():
    e = Engine()
    for r in _TRAIN_DATA_RESOLUTIONS:
        for c in _TRAIN_DATA_COARSENINGS:
            for model in ("jelly", "liquid", "snow"):
                x, material = make_train_sim_object(r, model)
                e.generate(
                    x,
                    material,
                    steps=_TRAIN_DATA_STEPS,
                    gmin=_TRAIN_DATA_GMIN,
                    gmax=_TRAIN_DATA_GMAX,
                    incr=_TRAIN_DATA_GINCR,
                    coarsening=c,
                )


# Make the testing-test cubes in the simulation
def make_test_sim_object(res: int):
    bounds1 = (0.4, 0.6)
    x1 = cube(bounds1, res=res)
    x1[:, 1] += 0.25
    material1 = np.array(["jelly"] * len(x1))

    bounds2 = (0.4, 0.6)
    x2 = cube(bounds2, res=res)
    material2 = np.array(["snow"] * len(x2))

    bounds3 = (0.4, 0.6)
    x3 = cube(bounds3, res=res)
    x3[:, 1] -= 0.25
    material3 = np.array(["liquid"] * len(x3))

    cubes = np.concatenate((x1, x2, x3))
    materials = np.concatenate((material1, material2, material3))

    return cubes, materials


def generate_all_test_datasets():
    e = Engine()
    resolution = 500
    x, material = make_test_sim_object(resolution)
    e.generate(
        x,
        material,
        steps=_TEST_DATA_STEPS,
        gmin=_TEST_DATA_GMIN,
        gmax=_TEST_DATA_GMAX,
        coarsening=_TEST_DATA_COARSENING,
    )

In [ ]:
"""
Load datasets and pre-process them
"""

def select_and_load_dataset(r: int, c: int):
    files = list(filter(lambda x: x.endswith(".npz"), os.listdir("old_datasets/")))
    files = list(filter(lambda f: f"res_{r}" in f and f"coarse_{c}" in f, files))
    files = list(map(lambda x: f"old_datasets/{x}", files))
    jelly = [f for f in files if "jelly" in f][0]
    snow = [f for f in files if "snow" in f][0]
    liquid = [f for f in files if "liquid" in f][0]

    return np.load(jelly), np.load(snow), np.load(liquid)

def make_inputs_and_targets(ds: np.ndarray):
    m_ini = (ds["initial"][:, :, :, 0] != 0) | (ds["initial"][:, :, :, 1] != 0)
    m_fin = (ds["final"][:, :, :, 0] != 0) | (ds["final"][:, :, :, 1] != 0)
    m_ini = np.expand_dims(m_ini, axis=3)
    m_fin = np.expand_dims(m_fin, axis=3)
    return [
        np.concatenate((ds["initial"], m_ini), axis=3),
        np.concatenate((ds["final"], m_fin), axis=3),
    ]

def transform(ds: np.ndarray):
    ds[0] /= 140
    ds[1] /= 140
    ds[0][:, :, :, 2] *= 140
    ds[1][:, :, :, 2] *= 140

def load_train_datasets_by_res_and_coarseness(res: int, coarseness: int):
    dataset_jelly, dataset_snow, dataset_liquid = select_and_load_dataset(res, coarseness)
    dataset_jelly = make_inputs_and_targets(dataset_jelly)
    dataset_snow = make_inputs_and_targets(dataset_snow)
    dataset_liquid = make_inputs_and_targets(dataset_liquid)

    transform(dataset_jelly)
    transform(dataset_snow)
    transform(dataset_liquid)

    return dataset_jelly, dataset_snow, dataset_liquid


In [ ]:
"""
Model Training
"""


def infer_model_name(expo):
    size = "86k"
    if expo == 4:
        size = "330k"
    elif expo == 5:
        size = "1_3m"
    elif expo == 6:
        size = "5_4m"
    elif expo == 7:
        size = "21m"
    return size


def train_model(X, y, name="", expo=_EXPO, dropout=_DROPOUT, lr=_LR, save=True):
    early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
    nn = make_model(expo=expo, dropout=dropout, lr=lr)
    print("Neural network parameters: ", nn.count_params())
    history = nn.fit(
        X,
        y,
        epochs=_EPOCHS,
        batch_size=_BATCH_SIZE,
        validation_split=_VALIDATION_SPLIT,
        callbacks=[early_stopping],
    )

    if save:
        if name == "":
            name = f"network_{infer_model_name(expo)}_lr_{lr}"
        print(f"Saving neural network to {name}.h5")
        nn.save(f"{name}.h5")

        print(f"Saving history to history_{name}.pickle")
        with open(f"history_{name}.pickle", "wb+") as f:
            pickle.dump(history, f)
    return history

def train_all_models_by_size():
    for expo in (3, 4, 5, 6, 7):
        for r in (400, 800, 1200):
            for c in (10, 15, 20):
                j, s, l = load_train_datasets_by_res_and_coarseness(r, c)
                X = np.concatenate((j[0], s[0], l[0]))
                y = np.concatenate((j[1], s[1], l[1]))
                name = f"network_{infer_model_name(expo)}_res_{r}_coarse_{c}_lr_{_LR}"
                train_model(X, y, name, expo=expo, dropout=_DROPOUT, lr=_LR, save=True)

def train_all_models_by_learning_rate():
    expo = 7
    r = 1200
    c = 10
    for lr in (0.1, 0.01, 0.001, 0.0001, 0.00001):
        j, s, l = load_train_datasets_by_res_and_coarseness(r, c)
        X = np.concatenate((j[0], s[0], l[0]))
        y = np.concatenate((j[1], s[1], l[1]))
        name = f"network_{infer_model_name(expo)}_res_{r}_coarse_{c}_lr_{lr}"
        train_model(X, y, name, expo=expo, dropout=_DROPOUT, lr=_LR, save=True)


def make_model(*, input_shape=_INPUT_SHAPE, expo=_EXPO, dropout=_DROPOUT, lr=_LR):
    def conv_block(
        name, filters, kernel_size=4, pad="same", t=False, act="relu", bn=True
    ):
        """High-level helper function to generate our block of convolutions.

        Args:
            name (str): The name of the block
            filters (int): The number of filters to apply.
            kernel_size (int, optional): The kernel size. Defaults to 4.
            pad (str, optional): The type of padding to use. Defaults to "same".
            t (bool, optional): Whether or not to transpose. Defaults to False.
            act (str, optional): The activation function to use. Defaults to "relu".
            bn (bool, optional): Whether or not to use batch norm. Defaults to True.

        Returns:
            Sequential: The sequential block.
        """
        block = Sequential(name=name)

        if act == "relu":
            block.add(L.ReLU())
        elif act == "leaky_relu":
            block.add(L.LeakyReLU(0.2))

        if not t:
            block.add(
                L.Conv2D(
                    filters,
                    kernel_size=kernel_size,
                    strides=(2, 2),
                    padding=pad,
                    use_bias=True,
                    activation=None,
                    kernel_initializer=RandomNormal(0.0, 0.2),
                )
            )
        else:
            block.add(L.UpSampling2D(interpolation="bilinear"))
            block.add(
                L.Conv2DTranspose(
                    filters=filters,
                    kernel_size=kernel_size - 1,
                    padding=pad,
                    activation=None,
                )
            )

        if dropout > 0:
            block.add(L.SpatialDropout2D(dropout))

        if bn:
            block.add(L.BatchNormalization(axis=-1, epsilon=1e-05, momentum=0.9))

        return block

    channels = int(2**expo + 0.5)
    e0 = Sequential(name="enc_0")
    e0.add(
        L.Conv2D(
            filters=_CHANNELS,
            kernel_size=4,
            strides=(2, 2),
            padding="same",
            activation=None,
            data_format="channels_last",
        )
    )
    e1 = conv_block("enc_1", channels, act="leaky_relu")
    e2 = conv_block("enc_2", channels * 2, act="leaky_relu")
    e3 = conv_block("enc_3", channels * 4, act="leaky_relu")
    e4 = conv_block("enc_4", channels * 8, act="leaky_relu", kernel_size=2, pad="valid")
    e5 = conv_block("enc_5", channels * 8, act="leaky_relu", kernel_size=2, pad="valid")

    dec_5 = conv_block("dec_5", channels * 8, t=True, kernel_size=2, pad="valid")
    dec_4 = conv_block("dec_4", channels * 8, t=True, kernel_size=2, pad="valid")
    dec_3 = conv_block("dec_3", channels * 4, t=True)
    dec_2 = conv_block("dec_2", channels * 2, t=True)
    dec_1 = conv_block("dec_1", channels, act=None, t=True, bn=False)
    dec_0 = Sequential(name="dec_0")
    dec_0.add(L.ReLU())
    dec_0.add(L.Conv2DTranspose(_CHANNELS, kernel_size=4, strides=(2, 2), padding="same"))

    # Forward Pass
    inputs = Input(shape=input_shape)
    out0 = e0(inputs)
    out1 = e1(out0)
    out2 = e2(out1)
    out3 = e3(out2)
    out4 = e4(out3)
    out5 = e5(out4)

    dout5 = dec_5(out5)
    dout5_out4 = tf.concat([dout5, out4], axis=3)
    dout4 = dec_4(dout5_out4)
    dout4_out3 = tf.concat([dout4, out3], axis=3)
    dout3 = dec_3(dout4_out3)
    dout3_out2 = tf.concat([dout3, out2], axis=3)
    dout2 = dec_2(dout3_out2)
    dout2_out1 = tf.concat([dout2, out1], axis=3)
    dout1 = dec_1(dout2_out1)
    dout1_out0 = tf.concat([dout1, out0], axis=3)
    dout0 = dec_0(dout1_out0)

    model = Model(inputs=inputs, outputs=dout0)
    model.compile(optimizer=Adam(lr, beta_1=0.5), loss=_LOSS)
    return model


In [ ]:
"""
Computes the results of the model
"""


def splice_label(name: str):
    fname_split = name.split("_")
    label = ".".join(
        fname_split[fname_split.index("network") + 1 : fname_split.index("res")]
    )
    return label


def plot_loss(histories: List[Tuple[str, History]], title: str, val="val_loss"):
    for label, history in histories:
        hd = history.history
        losses = hd[val]
        epochs = range(1, len(losses) + 1)
        plt.plot(epochs, losses, label=label)

    plt.title(title, fontsize=16)
    plt.xlabel("Epochs", fontsize=16)
    plt.ylabel("Loss", fontsize=16)
    plt.legend()
    plt.show()


def make_label_and_history_for_loss_comparison_plot(histories: List[str]):
    def sort_by_lr(ls: List[str]) -> List[str]:
        ret = [None] * 4
        for f in ls:
            if "lr_0.01" in f:
                ret[0] = f
            if "lr_0.001" in f:
                ret[1] = f
            if "lr_0.0001" in f:
                ret[2] = f
            if "lr_1e-05" in f:
                ret[3] = f
        return ret

    def make_label_and_plot_data(files: List[str]):
        for i, f in enumerate(files):
            res = f[f.index("lr") : f.index("scaled")]
            label = " ".join(res.split("_"))
            files[i] = (label, pickle.load(open(f, "rb")))

    histories = sort_by_lr(histories)
    make_label_and_plot_data(histories)
    return histories


def make_label_and_history_for_model_size_comparison_plot(histories: List[str]):
    def make_label_and_plot_data(files: List[str]):
        for i, f in enumerate(files):
            label = splice_label(f)
            files[i] = (label, pickle.load(open(f, "rb")))

    def sort_by_size(files: Tuple[str, Any]):
        _map = {
            "21m": 0,
            "5.4m": 1,
            "1.3m": 2,
            "330k": 3,
            "86k": 4,
        }
        ret = [None] * 5
        for f in files:
            ret[_map[f[0]]] = f
        return ret

    make_label_and_plot_data(histories)
    histories = sort_by_size(histories)
    return histories


def make_label_and_history_for_coarseness_comparison(histories: List[str], c: int):
    def make_label_and_plot_data(files: List[str]):
        for i, f in enumerate(files):
            res = " ".join(f[f.index("res_") : f.index("coarse_") - 1].split("_"))
            files[i] = (res.capitalize(), pickle.load(open(f, "rb")))

    def sort_by_resolution(files: List[str]) -> List[str]:
        ret = [None] * 3
        for f in files:
            if "400" in f:
                ret[0] = f
            elif "800" in f:
                ret[1] = f
            elif "1200" in f:
                ret[2] = f
        return ret

    histories = sort_by_resolution(list(filter(lambda x: f"coarse_{c}" in x, histories)))
    make_label_and_plot_data(histories)
    return histories


def plot_validation_loss_by_learning_rate():
    histories = list(
        filter(
            lambda x: x.startswith("history_")
            and "21m" in x
            and "coarse_10" in x
            and "res_1200" in x,
            os.listdir("."),
        )
    )

    plt.ylim(0.0, 0.0040)
    plot_loss(
        make_label_and_history_for_loss_comparison_plot(histories),
        "Validation Loss by Learning Rate",
    )


def plot_validation_loss_by_model_size():
    histories = list(
        filter(
            lambda x: x.startswith("history_")
            and "lr_0.0001" in x
            and "coarse_10" in x
            and "res_1200" in x,
            os.listdir("."),
        )
    )

    plt.ylim(0, 0.0004)
    plot_loss(
        make_label_and_history_for_model_size_comparison_plot(histories),
        "Validation Loss for All Model Sizes",
    )


def plot_validation_loss_by_coarseness(c: int):
    histories = list(
        filter(
            lambda x: x.startswith("history_") and "lr_0.0001" in x and "21m" in x,
            os.listdir("."),
        )
    )

    plot_loss(
        make_label_and_history_for_coarseness_comparison(histories, c),
        f"Validation Loss for Coarseness {c}",
    )


def plot_sbs(c: int, dataset: np.ndarray, models: List[Model]):
    steps_per_iter = _TEST_DATA_STEPS // c

    def convert_x_label(params: int):
        if params == 86262:
            label = "86k"
        elif params == 339014:
            label = "330k"
        elif params == 1_344_870:
            label = "1.3m"
        elif params == 5_357_990:
            label = "5.4m"
        elif params == 21_389_862:
            label = "21m"
        return label + " params"

    fig, axes = plt.subplots(len(models) + 1, 3, figsize=(30, 30))
    plt.subplots_adjust(bottom=0.2, top=0.8, right=0.65, left=0.35)
    slice_ = steps_per_iter // 2
    for i, axislist in enumerate(axes):
        if i == 0:
            prediction = dataset[1][slice_]
            xlabel = "Ground Truth"
        else:
            model = models[i - 1]
            xlabel = convert_x_label(model.count_params())
            prediction = model.predict(np.expand_dims(dataset[1][slice_], axis=0))
            prediction = np.squeeze(prediction)
        for j, axis in enumerate(axislist):
            axis.set_xticks([])
            axis.set_yticks([])
            axis.set_ylabel(_Y_AXIS_LABELS_GROUND_TRUTH[j], fontsize=14)
            axis.set_xlabel(xlabel, fontsize=14)

            im = axis.imshow(prediction[:, :, j], cmap=_CMAP)


In [ ]:
"""
Error calculation
"""

def compute_cumulative_error(model: Model):
    loss = Huber()
    dataset = np.load("dataset_res_1500_gmin_-100_to_gmax_-99_coarse_10_all.npz")

    dataset = make_inputs_and_targets(dataset)
    transform(dataset)

    dsini = dataset[0]
    dsfin = dataset[1]

    cum_error = 0.0
    for inp, gt in zip(dsini, dsfin):
        ypred = np.squeeze(model.predict(np.expand_dims(inp, axis=0)))
        cum_error += loss(gt * 140, ypred * 140).numpy()

    return cum_error / dsini.shape[0]